In [2]:
# #파이썬으로 gz 파일 압축 풀기
# import tarfile
# with tarfile.open(r'E:\Programming\python\aclImdb_v1\aclImdb_v1.tar.gz', 'r:gz') as tar:
#     tar.extractall()

In [1]:
import pyprind
import pandas as pd
import os

basepath = r'E:\Programming\python\aclImdb_v1\aclImdb'

##영화 리뷰를 한 줄씩 읽어서 데이터 프레임화 하기
labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:25


In [3]:
import numpy as np

#클래스 레이블 순서대로 데이터가 나열되있으므로 permutation 함수로 데이터 섞기
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv(r'E:\Programming\python\Machine-Learning\data\movie_data.csv', index=False, encoding='utf-8')

In [4]:
#데이터 csv파일로 저장하기
df = pd.read_csv(r'E:\Programming\python\Machine-Learning\data\movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


In [5]:
#데이터 프레임 행과 열의 개수 확인하기
df.shape

(50000, 2)

In [5]:
####Bag of Word####
#1. 전체 문서에 대해 고유한 토큰(예컨데, 단어로 이루어진 어휘 사전) 생성
#2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터 생성

In [6]:
###BoW 만들기
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #CountVectorizer은 문서 또는 문장으로 이루어진 텍스트 데이터 배열을 입력 받아 BoW 모델 생성

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [7]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [8]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [9]:
###tf(단어 빈도)-idf(역문서 빈도)###
#특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [11]:
##불필요한 문자 제거를 통한 텍스트 정제

In [12]:
#랜덤하게 섞은 데이터셋의 첫 번째 문서에서 마지막 50개의 문자를 출력
df.loc[0, 'review'][-50:] #HTML 마크업 문자와 글자가 아닌 문자 또한 포함되어있음

'and I suggest that you go see it before you judge.'

In [13]:
##이모티콘은 정보가 될수 있으므로 제외하고 그 외의 모든 구두점 문자 제거
#정규표현식 사용
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #html 마크업을 삭제하는 정규표현식
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [14]:
preprocessor(df.loc[0, 'review'][-50:])

'and i suggest that you go see it before you judge '

In [15]:
df['review'] = df['review'].apply(preprocessor)

In [16]:
##공백 문자를 기준으로 단어를 나누기
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [17]:
##nltke의 어간추출 알고리즘
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [18]:
##불용어##
#모든 텍스트에 흔하게 등장하는 단어
#tf-idf 보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 유용
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
#불용어 적용하기
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [20]:
##영화 리뷰를 긍정과 부정 리뷰로 분리하는 로지스틱 회귀 모델 훈련 시키기##
#1. 정제된 텍스트 문서가 저장된 dataframe을 2만 5천개는 훈련 세트, 2만 5천개는 테스트 세트로 나누기

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [24]:
#GridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 최적의 매개변수 조함 확인
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

#CountVectorizer와 TfidfTransformer을 합친 기능인 TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],          ##TfidfVectorizer의 기본 매개 변수를 셋팅하여 tf-idf를 계산
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}, 
              {'vect__ngram_range': [(1,1)], 
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'vect__use_idf': [False],              ##단어 빈도를 사용해 모델을 훈련 시키기 위해 좌와 같이 지정함.
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
              ]
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(solver='liblinear', random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 98.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 135.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [26]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_) ##책의 최적의 조합은 C =10.0, L2규제 사용

최적의 매개변수 조합: {'clf__C': 1.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x000002A1C2130798>} 


In [27]:
print('CV 정확도: %.3f' %gs_lr_tfidf.best_score_)  ##책은 0.897

CV 정확도: 0.876


In [28]:
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))  ##cordms 0.899

테스트 정확도: 0.877


In [29]:
##외부 메모리 학습 기법##

In [6]:
#tokenizer 함수를 만들어 movie_data.csv 파일로 부터 읽은 텍스트 데이터를 정제하고 불용어를 제외한 다음에 단어 토큰으로 분리
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
           + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

#한 번에 문서 하나씩 읽어서 반환하는 함수
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) #헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [7]:
next(stream_docs(path=r'E:\Programming\python\Machine-Learning\data\movie_data.csv'))

('"Election is a Chinese mob movie, or triads in this case. Every two years an election is held to decide on a new leader, and at first it seems a toss up between Big D (Tony Leung Ka Fai, or as I know him, ""The Other Tony Leung"") and Lok (Simon Yam, who was Judge in Full Contact!). Though once Lok wins, Big D refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader. Unlike any other Asian film I watch featuring gangsters, this one is not an action movie. It has its bloody moments, when necessary, as in Goodfellas, but it\'s basically just a really effective drama. There are a lot of characters, which is really hard to keep track of, but I think that plays into the craziness of it all a bit. A 100-year-old baton, which is the symbol of power I mentioned before, changes hands several times before things settle down. And though it may appear that the film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Si

In [8]:
#size 매개변수에서 지정한만큼 문서를 반환하는 함수
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y


In [9]:
##CountVectorizer나 TfidfVectorizer는 전체 어휘사전을 메모리에 가지고있어야 하기에 사용이 불가.
##대신 HashingVectorizer를 사용
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',    ##HashingVectorizer 초기화하기
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)  ##로지스틱 모델 초기화하기(log를 사용해서))
doc_stream = stream_docs(path=r'E:\Programming\python\Machine-Learning\data\movie_data.csv')

In [10]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


In [11]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))  ##책은 0.867

정확도: 0.802


In [12]:
clf = clf.partial_fit(X_test, y_test)

In [38]:
##토픽 모델링##

In [39]:
####잠재 디클레이 할당####

In [41]:
df = pd.read_csv(r'E:\Programming\python\Machine-Learning\data\movie_data.csv', encoding='utf-8')

In [42]:
#BoW만들기
count = CountVectorizer(stop_words='english',
                        max_df=.1,  #단어의 최대 문서 빈도를 10%로 지정 => 여러 문서에 너무 자주 등장하는 단어 제외
                        max_features=5000) #가장 자주 등장하는 단어 수 제한 => LDA 추론 성능 향상
X = count.fit_transform(df['review'].values)

In [43]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, #추정할 토픽의 개수
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [44]:
lda.components_.shape

(10, 5000)

In [45]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print('토픽 %d:' % (topic_idx + 1))
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words -1:-1]]))

토픽 1:
worst minutes awful script stupid
토픽 2:
family mother father children girl
토픽 3:
american war dvd music tv
토픽 4:
human audience cinema art sense
토픽 5:
police guy car dead murder
토픽 6:
horror house sex girl woman
토픽 7:
role performance comedy actor performances
토픽 8:
series episode war episodes tv
토픽 9:
book version original read novel
토픽 10:
action fight guy guys cool


In [47]:
#공포 토픽(토픽 6)확인하기
horror = X_topics[:, 5].argsort()[::-1]  ##토픽6이므로 인덱스는 5이다
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [15]:
###############################
###############################
########9장 모델 저장하기#######
###############################
###############################

In [ ]:
####학습된 사이킷런 추정기 저장####

##pickle 모듈 사용
##-파이썬 객체의 구조를 압축된 바이트코드로 직렬화하고 복원할 수 있다

In [16]:
import pickle
import os

##저장할 디렉토리 만들기##
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

##불용어 저장하기
pickle.dump(stop,  #저장할 객체 받기
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'),  #wb는 이진모드로 파일을 연다는 의미
            protocol=4) #파이썬 3.4버전과 그 이상에서 사용할 수 있는 최신의 pickle 프로토콜 사용

##훈련된 로지스틱 회귀 모델 저장하기
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)

In [17]:
##HashinVectorizer는 학습 과정이 없기에 pickle로 직렬화 할 필요없다.